# Layers API


In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

## PREPARE DATA

### WINE DATASET

https://archive.ics.uci.edu/ml/datasets/wine

In [2]:
# Load data
wine_data = load_wine()

In [3]:
# Keys in dataset
wine_data.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names'])

In [4]:
# Dataset info
print(wine_data.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

In [5]:
# Grab features
feat_data = wine_data['data']

In [6]:
# Grab labels
labels = wine_data['target']

In [7]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(feat_data,
                                                    labels,
                                                    test_size=0.3,
                                                   random_state=101)

### Scale the Data

In [8]:
# Transforms features by scaling each feature to a given range.
scaler = MinMaxScaler()

In [9]:
# Scaled train data
scaled_x_train = scaler.fit_transform(X_train)

In [10]:
# Scaled test data
scaled_x_test = scaler.transform(X_test)

## CREATE MODEL

In [11]:
# Labels to One-Hot Encode vector
one_hot_y_train = pd.get_dummies(y_train).values
one_hot_y_test = pd.get_dummies(y_test).values

In [12]:
# Number of features
num_feat = 13
# Number of neurons in layer 1
num_hidden1 = 13
# Number of neurons in layer 2
num_hidden2 = 13
# Number of neurons in output layer
num_outputs = 3
# Learning rate
learning_rate = 0.01

### Placeholder

In [13]:
# Placeholders
X = tf.placeholder(tf.float32,shape=[None,num_feat])
y_true = tf.placeholder(tf.float32,shape=[None,3])

In [14]:
# Activation function
actf = tf.nn.relu

In [15]:
# Layer 1
hidden1 = fully_connected(X,num_hidden1,activation_fn=actf)

In [16]:
# Layer 2
hidden2 = fully_connected(hidden1,num_hidden2,activation_fn=actf)

In [17]:
# Output layer
output = fully_connected(hidden2,num_outputs)

In [18]:
# Loss function
loss = tf.losses.softmax_cross_entropy(onehot_labels=y_true, logits=output)

In [19]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
# Minimize error
train = optimizer.minimize(loss)

In [20]:
# Initializer
init = tf.global_variables_initializer()

In [21]:
# Number of epochs
epochs = 1000

# Create session
with tf.Session() as sess:
    
    # Run initializer
    sess.run(init)
    
    # Loop for training
    for i in range(epochs):
        
        # Run training
        sess.run(train,feed_dict={X:scaled_x_train,y_true: one_hot_y_train})
        
    # Output values
    logits = output.eval(feed_dict={X:scaled_x_test})
    
    # Predictions
    preds = tf.argmax(logits,axis=1)
    
    # Results
    results = preds.eval()

In [22]:
# Show metrics for test dataset
print(classification_report(results,y_test))

              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       0.95      1.00      0.98        21
           2       1.00      1.00      1.00        13

   micro avg       0.98      0.98      0.98        54
   macro avg       0.98      0.98      0.98        54
weighted avg       0.98      0.98      0.98        54

